In [1]:
%matplotlib inline
import dgl
import copy
import glob
import pprint
import numpy as np
import torch
import torch.nn as nn
import awkward as ak
import networkx as nx
import matplotlib.pyplot as plt
from os import path
from pathlib import Path
from trainresults import TrainResults
from train_eval_func import train, evaluate
from copy import deepcopy
from dgl.data import DGLDataset
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from TauGraphDatasetInfo import TauGraphDatasetInfo
from CustomGCNs import CustomGCN_OnlyNFeatSumMsg, CustomGCN_OnlyNFeatMeanMsg
from TauGraphDataset import TauGraphDataset, GetNodeFeatureVectors, GetEdgeFeatureVectors, GetNeighborNodes, GetEdgeList

plt.rcParams.update({'font.size': 20})
plt.rcParams['text.usetex'] = True
lw = 2
xyLabelFontSize = 20
xLabelPad = 10
yLabelPad = 15
pp = pprint.PrettyPrinter()

Using backend: pytorch


In [2]:
# training
def trainEpochs(model, device, dataloader, optimizer, loss_fn, batchsize, nEpochs):
    results = TrainResults()
    results.startTrainingTimer()
    bestModel = None
    bestValAcc = 0.0

    for epoch in range(nEpochs):
        # train
        epochLoss = train(model, device, dataloader, optimizer, loss_fn, batchsize, results)

        # evaluate
        train_result = evaluate(model, device, train_dataloader, loss_fn)
        val_result = evaluate(model, device, val_dataloader, loss_fn)
        test_result = evaluate(model, device, test_dataloader, loss_fn)

        results.addEpochResult(epochLoss, train_result, val_result, test_result)
        results.printLastResult()

        if results.best_val_acc > bestValAcc:
            bestValAcc = results.best_val_acc
            bestModel = copy.deepcopy(model)
    
    results.endTrainingTimer()

    return results, bestModel

In [3]:
def getDatasetNames(datasetDir):
    files = glob.glob(datasetDir + '/*.json', recursive=True)
    files.sort()
    datasetDirectories = [path.dirname(file) for file in files]
    datasetnames = [path.normpath(dir).split(path.sep)[-1] for dir in datasetDirectories]
    return datasetDirectories, datasetnames

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

datasetDir = '/ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets'
datasetDirs, datasetNames = getDatasetNames(datasetDir)
print(datasetDirs)
print(datasetNames)

device: cpu
['/ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets']
['Graphs_DYJetsToLL_M-50_genuineTaus_and_jets']


datasetName = datasetNames[0]
datasetDir = datasetDirs[0]
dataset = TauGraphDataset(datasetName, datasetDir)
dataset.printProperties()

graph, label = dataset[0]
print(graph)
print(f'Label: {label}')
print(GetNodeFeatureVectors(graph))

In [5]:
import time
now = time.time()

paramList =[(16, 2, 0), # node_hiddenfeats, #num_messagepasses, #dropout
          (16, 2, 0.3),
          (16, 2, 0.5),
          (32, 2, 0),
          (32, 2, 0.3),
          (32, 2, 0.5),
          (64, 2, 0),
          (64, 2, 0.3),
          (64, 2, 0.5),
          (16, 3, 0), 
          (16, 3, 0.3),
          (16, 3, 0.5),
          (32, 3, 0),
          (32, 3, 0.3),
          (32, 3, 0.5),
          (64, 3, 0),
          (64, 3, 0.3),
          (64, 3, 0.5),
          (16, 5, 0), 
          (16, 5, 0.3),
          (16, 5, 0.5),
          (32, 5, 0),
          (32, 5, 0.3),
          (32, 5, 0.5),
          (64, 5, 0),
          (64, 5, 0.3),
          (64, 5, 0.5)]
          #[(16, 1, 0), 
          #(16, 1, 0.3),
          #(16, 1, 0.5),
          #(32, 1, 0),
          #(32, 1, 0.3),
          #(32, 1, 0.5),
          #(64, 1, 0),
          #(64, 1, 0.3),
          #(64, 1, 0.5),

batchSize = 1024
print(f'Device: {device}')

for n_hidden, msgpasses, dropout in paramList:
    for i in range(len(datasetDirs)):
        dataset = TauGraphDataset(datasetNames[i], datasetDirs[i])
        splitIndices = dataset.get_split_indices()

        train_sampler = SubsetRandomSampler(splitIndices['train'])
        val_sampler = SubsetRandomSampler(splitIndices['valid'])
        test_sampler = SubsetRandomSampler(splitIndices['test'])

        train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=batchSize, drop_last=False)
        val_dataloader = GraphDataLoader(dataset, sampler=val_sampler, batch_size=batchSize, drop_last=False)
        test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)

        # Create the model with given dimensions
        model = CustomGCN_OnlyNFeatSumMsg(dataset.dim_nfeats, n_hidden, dataset.num_graph_classes, msgpasses, dropout).to(device)
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.CrossEntropyLoss()
        epochs = 30

        # train
        print(f'Beginning training on dataset {datasetNames[i]}')
        results, bestmodel = trainEpochs(model, device, train_dataloader, optimizer, loss_fn, batchSize, epochs)
        results.printBestResult()

        # save results
        outputFolder = path.join(datasetDirs[i], f'Output_CustomGCN_OnlyNFeatSumMsg_NHiddenFeat_{n_hidden}_MsgPasses_{msgpasses}_Dropout_{dropout}')
        Path(outputFolder).mkdir(parents=True, exist_ok=True)

        results.savePlots(outputFolder)
        results.dumpSummary(outputFolder)
        results.pickledump(outputFolder)

        # save the best model for inference. (when loading for inference -> model.eval()!! )
        # https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
        torch.save(bestmodel.state_dict(), path.join(outputFolder, 'model.pt'))

        print(f'------------------({i+1}/{len(datasetDirs)}) models trained------------------\n')


end = time.time()
elapsed = end - now
print(f'{elapsed} seconds elapsed')

Device: cpu
Done loading data from cached files.
Beginning training on dataset Graphs_DYJetsToLL_M-50_genuineTaus_and_jets
Epoch: 0, Loss: 5.8228, Validation Loss: 0.5550, Train: 0.677, Validation: 0.675, Test: 0.678, AUC: 0.852
Epoch: 1, Loss: 0.4854, Validation Loss: 0.4341, Train: 0.834, Validation: 0.834, Test: 0.839, AUC: 0.920
Epoch: 2, Loss: 0.4070, Validation Loss: 0.3798, Train: 0.876, Validation: 0.877, Test: 0.881, AUC: 0.933
Epoch: 3, Loss: 0.3580, Validation Loss: 0.3361, Train: 0.887, Validation: 0.887, Test: 0.890, AUC: 0.945
Epoch: 4, Loss: 0.3235, Validation Loss: 0.3073, Train: 0.891, Validation: 0.891, Test: 0.895, AUC: 0.951
Epoch: 5, Loss: 0.3012, Validation Loss: 0.2885, Train: 0.894, Validation: 0.893, Test: 0.894, AUC: 0.953
Epoch: 6, Loss: 0.2865, Validation Loss: 0.2791, Train: 0.894, Validation: 0.894, Test: 0.895, AUC: 0.953
Epoch: 7, Loss: 0.2782, Validation Loss: 0.2749, Train: 0.894, Validation: 0.893, Test: 0.895, AUC: 0.953
Epoch: 8, Loss: 0.2751, Valid

Epoch: 10, Loss: 0.2886, Validation Loss: 0.2824, Train: 0.893, Validation: 0.894, Test: 0.894, AUC: 0.955
Epoch: 11, Loss: 0.2840, Validation Loss: 0.2801, Train: 0.890, Validation: 0.890, Test: 0.890, AUC: 0.955
Epoch: 12, Loss: 0.2809, Validation Loss: 0.2869, Train: 0.894, Validation: 0.893, Test: 0.898, AUC: 0.955
Epoch: 13, Loss: 0.2773, Validation Loss: 0.2716, Train: 0.896, Validation: 0.896, Test: 0.897, AUC: 0.956
Epoch: 14, Loss: 0.2754, Validation Loss: 0.2681, Train: 0.895, Validation: 0.895, Test: 0.896, AUC: 0.956
Epoch: 15, Loss: 0.2760, Validation Loss: 0.2696, Train: 0.897, Validation: 0.896, Test: 0.898, AUC: 0.956
Epoch: 16, Loss: 0.2753, Validation Loss: 0.2670, Train: 0.894, Validation: 0.894, Test: 0.895, AUC: 0.956
Epoch: 17, Loss: 0.2734, Validation Loss: 0.2715, Train: 0.891, Validation: 0.891, Test: 0.892, AUC: 0.956
Epoch: 18, Loss: 0.2740, Validation Loss: 0.2654, Train: 0.896, Validation: 0.896, Test: 0.897, AUC: 0.956
Epoch: 19, Loss: 0.2732, Validation L

Epoch: 21, Loss: 0.2758, Validation Loss: 0.2659, Train: 0.895, Validation: 0.894, Test: 0.896, AUC: 0.956
Epoch: 22, Loss: 0.2726, Validation Loss: 0.2838, Train: 0.879, Validation: 0.879, Test: 0.879, AUC: 0.956
Epoch: 23, Loss: 0.2728, Validation Loss: 0.2627, Train: 0.895, Validation: 0.894, Test: 0.896, AUC: 0.956
Epoch: 24, Loss: 0.2700, Validation Loss: 0.2756, Train: 0.895, Validation: 0.894, Test: 0.899, AUC: 0.956
Epoch: 25, Loss: 0.2706, Validation Loss: 0.2638, Train: 0.895, Validation: 0.894, Test: 0.896, AUC: 0.956
Epoch: 26, Loss: 0.2687, Validation Loss: 0.2722, Train: 0.888, Validation: 0.888, Test: 0.889, AUC: 0.956
Epoch: 27, Loss: 0.2687, Validation Loss: 0.2634, Train: 0.898, Validation: 0.898, Test: 0.900, AUC: 0.956
Epoch: 28, Loss: 0.2687, Validation Loss: 0.2632, Train: 0.896, Validation: 0.896, Test: 0.898, AUC: 0.956
Epoch: 29, Loss: 0.2705, Validation Loss: 0.2669, Train: 0.897, Validation: 0.897, Test: 0.899, AUC: 0.956
Best epoch: 
Epoch: 27, Loss: 0.2687,

------------------(1/1) models trained------------------

Done loading data from cached files.
Beginning training on dataset Graphs_DYJetsToLL_M-50_genuineTaus_and_jets
Epoch: 0, Loss: 1.3020, Validation Loss: 0.5973, Train: 0.663, Validation: 0.663, Test: 0.667, AUC: 0.948
Epoch: 1, Loss: 0.4898, Validation Loss: 0.5269, Train: 0.745, Validation: 0.745, Test: 0.748, AUC: 0.952
Epoch: 2, Loss: 0.3787, Validation Loss: 0.3697, Train: 0.861, Validation: 0.861, Test: 0.864, AUC: 0.952
Epoch: 3, Loss: 0.4812, Validation Loss: 0.4054, Train: 0.834, Validation: 0.835, Test: 0.842, AUC: 0.952
Epoch: 4, Loss: 0.3182, Validation Loss: 0.2988, Train: 0.895, Validation: 0.894, Test: 0.898, AUC: 0.951
Epoch: 5, Loss: 0.3067, Validation Loss: 0.3377, Train: 0.875, Validation: 0.876, Test: 0.879, AUC: 0.951
Epoch: 6, Loss: 0.3132, Validation Loss: 0.2873, Train: 0.889, Validation: 0.887, Test: 0.890, AUC: 0.951
Epoch: 7, Loss: 0.2951, Validation Loss: 0.2794, Train: 0.898, Validation: 0.898, Test: 0

Epoch: 9, Loss: 0.2899, Validation Loss: 0.2853, Train: 0.890, Validation: 0.890, Test: 0.892, AUC: 0.953
Epoch: 10, Loss: 0.2888, Validation Loss: 0.2835, Train: 0.894, Validation: 0.894, Test: 0.895, AUC: 0.953
Epoch: 11, Loss: 0.3288, Validation Loss: 0.3939, Train: 0.864, Validation: 0.864, Test: 0.866, AUC: 0.953
Epoch: 12, Loss: 0.2941, Validation Loss: 0.2789, Train: 0.892, Validation: 0.893, Test: 0.893, AUC: 0.953
Epoch: 13, Loss: 0.5579, Validation Loss: 0.2834, Train: 0.887, Validation: 0.888, Test: 0.888, AUC: 0.953
Epoch: 14, Loss: 0.2852, Validation Loss: 0.2801, Train: 0.892, Validation: 0.892, Test: 0.893, AUC: 0.953
Epoch: 15, Loss: 0.2854, Validation Loss: 0.2849, Train: 0.887, Validation: 0.887, Test: 0.888, AUC: 0.953
Epoch: 16, Loss: 0.2839, Validation Loss: 0.2785, Train: 0.890, Validation: 0.890, Test: 0.890, AUC: 0.953
Epoch: 17, Loss: 0.2843, Validation Loss: 0.2799, Train: 0.893, Validation: 0.894, Test: 0.896, AUC: 0.953
Epoch: 18, Loss: 0.2840, Validation Lo

...

Epoch: 9, Loss: 0.2967, Validation Loss: 0.2888, Train: 0.895, Validation: 0.895, Test: 0.897, AUC: 0.955
Epoch: 10, Loss: 0.3041, Validation Loss: 0.2861, Train: 0.888, Validation: 0.887, Test: 0.890, AUC: 0.955
Epoch: 11, Loss: 0.2904, Validation Loss: 0.2848, Train: 0.882, Validation: 0.883, Test: 0.884, AUC: 0.955
Epoch: 12, Loss: 0.2904, Validation Loss: 0.2906, Train: 0.877, Validation: 0.877, Test: 0.878, AUC: 0.955
Epoch: 13, Loss: 0.2919, Validation Loss: 0.2990, Train: 0.891, Validation: 0.892, Test: 0.894, AUC: 0.955
Epoch: 14, Loss: 0.2883, Validation Loss: 0.2767, Train: 0.891, Validation: 0.891, Test: 0.891, AUC: 0.955
Epoch: 15, Loss: 0.2928, Validation Loss: 0.2769, Train: 0.892, Validation: 0.892, Test: 0.893, AUC: 0.955
Epoch: 16, Loss: 0.2844, Validation Loss: 0.2731, Train: 0.893, Validation: 0.893, Test: 0.893, AUC: 0.955
Epoch: 17, Loss: 0.2856, Validation Loss: 0.2780, Train: 0.887, Validation: 0.887, Test: 0.887, AUC: 0.955
Epoch: 18, Loss: 0.2820, Validation Lo

Epoch: 20, Loss: 0.4408, Validation Loss: 0.9410, Train: 0.863, Validation: 0.862, Test: 0.868, AUC: 0.927
Epoch: 21, Loss: 0.4336, Validation Loss: 0.3799, Train: 0.887, Validation: 0.887, Test: 0.887, AUC: 0.949
Epoch: 22, Loss: 0.3647, Validation Loss: 0.3276, Train: 0.877, Validation: 0.879, Test: 0.877, AUC: 0.949
Epoch: 23, Loss: 0.3393, Validation Loss: 0.3273, Train: 0.878, Validation: 0.878, Test: 0.878, AUC: 0.949
Epoch: 24, Loss: 0.3518, Validation Loss: 0.3241, Train: 0.876, Validation: 0.876, Test: 0.875, AUC: 0.949
Epoch: 25, Loss: 0.3577, Validation Loss: 0.3287, Train: 0.878, Validation: 0.878, Test: 0.878, AUC: 0.949
Epoch: 26, Loss: 0.3486, Validation Loss: 0.3279, Train: 0.885, Validation: 0.884, Test: 0.886, AUC: 0.949
Epoch: 27, Loss: 0.3519, Validation Loss: 0.3250, Train: 0.875, Validation: 0.874, Test: 0.874, AUC: 0.949
Epoch: 28, Loss: 0.3283, Validation Loss: 0.3220, Train: 0.874, Validation: 0.874, Test: 0.874, AUC: 0.949
Epoch: 29, Loss: 0.3673, Validation L

------------------(1/1) models trained------------------

Done loading data from cached files.
Beginning training on dataset Graphs_DYJetsToLL_M-50_genuineTaus_and_jets
Epoch: 0, Loss: 48.6779, Validation Loss: 98.3509, Train: 0.570, Validation: 0.571, Test: 0.572, AUC: 0.729
Epoch: 1, Loss: 35.7272, Validation Loss: 5.4740, Train: 0.767, Validation: 0.763, Test: 0.767, AUC: 0.888
Epoch: 2, Loss: 8.5091, Validation Loss: 3.4020, Train: 0.808, Validation: 0.805, Test: 0.814, AUC: 0.916
Epoch: 3, Loss: 14.5094, Validation Loss: 21.5158, Train: 0.684, Validation: 0.688, Test: 0.688, AUC: 0.916
Epoch: 4, Loss: 15.0442, Validation Loss: 13.3333, Train: 0.691, Validation: 0.692, Test: 0.694, AUC: 0.916
Epoch: 5, Loss: 6.6834, Validation Loss: 2.0228, Train: 0.831, Validation: 0.831, Test: 0.839, AUC: 0.933
Epoch: 6, Loss: 7.4121, Validation Loss: 9.6733, Train: 0.767, Validation: 0.766, Test: 0.772, AUC: 0.933
Epoch: 7, Loss: 3.3386, Validation Loss: 2.8686, Train: 0.832, Validation: 0.832, 

Epoch: 9, Loss: 3.5314, Validation Loss: 0.5183, Train: 0.784, Validation: 0.786, Test: 0.786, AUC: 0.943
Epoch: 10, Loss: 1.1390, Validation Loss: 0.4576, Train: 0.813, Validation: 0.813, Test: 0.815, AUC: 0.943
Epoch: 11, Loss: 2.2816, Validation Loss: 1.3975, Train: 0.500, Validation: 0.499, Test: 0.502, AUC: 0.943
Epoch: 12, Loss: 1.3183, Validation Loss: 0.6381, Train: 0.728, Validation: 0.730, Test: 0.729, AUC: 0.943
Epoch: 13, Loss: 0.8282, Validation Loss: 0.3789, Train: 0.848, Validation: 0.849, Test: 0.850, AUC: 0.943
Epoch: 14, Loss: 0.6360, Validation Loss: 0.3771, Train: 0.847, Validation: 0.849, Test: 0.849, AUC: 0.943
Epoch: 15, Loss: 0.5837, Validation Loss: 0.3706, Train: 0.848, Validation: 0.850, Test: 0.851, AUC: 0.943
Epoch: 16, Loss: 2.2608, Validation Loss: 0.9795, Train: 0.500, Validation: 0.499, Test: 0.502, AUC: 0.943
Epoch: 17, Loss: 1.1526, Validation Loss: 0.8341, Train: 0.500, Validation: 0.499, Test: 0.502, AUC: 0.943
Epoch: 18, Loss: 0.9304, Validation Lo

Epoch: 20, Loss: 0.3518, Validation Loss: 0.3385, Train: 0.858, Validation: 0.859, Test: 0.862, AUC: 0.943
Epoch: 21, Loss: 0.3474, Validation Loss: 0.3366, Train: 0.858, Validation: 0.859, Test: 0.862, AUC: 0.943
Epoch: 22, Loss: 0.3479, Validation Loss: 0.3374, Train: 0.858, Validation: 0.859, Test: 0.862, AUC: 0.943
Epoch: 23, Loss: 0.3442, Validation Loss: 0.3346, Train: 0.858, Validation: 0.859, Test: 0.862, AUC: 0.943
Epoch: 24, Loss: 0.3460, Validation Loss: 0.3397, Train: 0.858, Validation: 0.859, Test: 0.862, AUC: 0.943
Epoch: 25, Loss: 0.3443, Validation Loss: 0.3338, Train: 0.858, Validation: 0.859, Test: 0.862, AUC: 0.943
Epoch: 26, Loss: 0.3426, Validation Loss: 0.3335, Train: 0.858, Validation: 0.859, Test: 0.862, AUC: 0.943
Epoch: 27, Loss: 0.3415, Validation Loss: 0.3353, Train: 0.858, Validation: 0.859, Test: 0.862, AUC: 0.943
Epoch: 28, Loss: 0.3415, Validation Loss: 0.3359, Train: 0.858, Validation: 0.859, Test: 0.862, AUC: 0.943
Epoch: 29, Loss: 0.3405, Validation L

In [6]:
import time
now = time.time()

paramList = [(16, 1, 0), # node_hiddenfeats, #num_messagepasses, #dropout
          (16, 1, 0.3),
          (16, 1, 0.5),
          (32, 1, 0),
          (32, 1, 0.3),
          (32, 1, 0.5),
          (64, 1, 0),
          (64, 1, 0.3),
          (64, 1, 0.5),
          (16, 2, 0), 
          (16, 2, 0.3),
          (16, 2, 0.5),
          (32, 2, 0),
          (32, 2, 0.3),
          (32, 2, 0.5),
          (64, 2, 0),
          (64, 2, 0.3),
          (64, 2, 0.5),
          (16, 3, 0), 
          (16, 3, 0.3),
          (16, 3, 0.5),
          (32, 3, 0),
          (32, 3, 0.3),
          (32, 3, 0.5),
          (64, 3, 0),
          (64, 3, 0.3),
          (64, 3, 0.5),
          (16, 5, 0), 
          (16, 5, 0.3),
          (16, 5, 0.5),
          (32, 5, 0),
          (32, 5, 0.3),
          (32, 5, 0.5),
          (64, 5, 0),
          (64, 5, 0.3),
          (64, 5, 0.5)]

batchSize = 1024
print(f'Device: {device}')

for n_hidden, msgpasses, dropout in paramList:
    for i in range(len(datasetDirs)):
        dataset = TauGraphDataset(datasetNames[i], datasetDirs[i])
        splitIndices = dataset.get_split_indices()

        train_sampler = SubsetRandomSampler(splitIndices['train'])
        val_sampler = SubsetRandomSampler(splitIndices['valid'])
        test_sampler = SubsetRandomSampler(splitIndices['test'])

        train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=batchSize, drop_last=False)
        val_dataloader = GraphDataLoader(dataset, sampler=val_sampler, batch_size=batchSize, drop_last=False)
        test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)

        # Create the model with given dimensions
        model = CustomGCN_OnlyNFeatMeanMsg(dataset.dim_nfeats, n_hidden, dataset.num_graph_classes, msgpasses, dropout).to(device)
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.CrossEntropyLoss()
        epochs = 30

        # train
        print(f'Beginning training on dataset {datasetNames[i]}')
        results, bestmodel = trainEpochs(model, device, train_dataloader, optimizer, loss_fn, batchSize, epochs)
        results.printBestResult()

        # save results
        outputFolder = path.join(datasetDirs[i], f'Output_CustomGCN_OnlyNFeatMeanMsg_NHiddenFeat_{n_hidden}_MsgPasses_{msgpasses}_Dropout_{dropout}')
        Path(outputFolder).mkdir(parents=True, exist_ok=True)

        results.savePlots(outputFolder)
        results.dumpSummary(outputFolder)
        results.pickledump(outputFolder)

        # save the best model for inference. (when loading for inference -> model.eval()!! )
        # https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
        torch.save(bestmodel.state_dict(), path.join(outputFolder, 'model.pt'))

        print(f'------------------({i+1}/{len(datasetDirs)}) models trained------------------\n')


end = time.time()
elapsed = end - now
print(f'{elapsed} seconds elapsed')

Device: cpu
Done loading data from cached files.
Beginning training on dataset Graphs_DYJetsToLL_M-50_genuineTaus_and_jets
Epoch: 0, Loss: 0.6854, Validation Loss: 0.6646, Train: 0.613, Validation: 0.612, Test: 0.612, AUC: 0.659
Epoch: 1, Loss: 0.6530, Validation Loss: 0.6413, Train: 0.645, Validation: 0.644, Test: 0.647, AUC: 0.713
Epoch: 2, Loss: 0.6303, Validation Loss: 0.6214, Train: 0.673, Validation: 0.672, Test: 0.673, AUC: 0.746
Epoch: 3, Loss: 0.6083, Validation Loss: 0.5965, Train: 0.696, Validation: 0.694, Test: 0.696, AUC: 0.776
Epoch: 4, Loss: 0.5854, Validation Loss: 0.5743, Train: 0.721, Validation: 0.720, Test: 0.722, AUC: 0.799
Epoch: 5, Loss: 0.5623, Validation Loss: 0.5505, Train: 0.740, Validation: 0.736, Test: 0.741, AUC: 0.827
Epoch: 6, Loss: 0.5390, Validation Loss: 0.5247, Train: 0.756, Validation: 0.754, Test: 0.757, AUC: 0.846
Epoch: 7, Loss: 0.5131, Validation Loss: 0.4983, Train: 0.770, Validation: 0.770, Test: 0.773, AUC: 0.867
Epoch: 8, Loss: 0.4924, Valid

Epoch: 10, Loss: 0.4594, Validation Loss: 0.4530, Train: 0.784, Validation: 0.786, Test: 0.785, AUC: 0.885
Epoch: 11, Loss: 0.4554, Validation Loss: 0.4481, Train: 0.786, Validation: 0.788, Test: 0.786, AUC: 0.887
Epoch: 12, Loss: 0.4523, Validation Loss: 0.4464, Train: 0.791, Validation: 0.792, Test: 0.792, AUC: 0.889
Epoch: 13, Loss: 0.4501, Validation Loss: 0.4449, Train: 0.791, Validation: 0.793, Test: 0.790, AUC: 0.890
Epoch: 14, Loss: 0.4485, Validation Loss: 0.4424, Train: 0.796, Validation: 0.797, Test: 0.797, AUC: 0.891
Epoch: 15, Loss: 0.4472, Validation Loss: 0.4438, Train: 0.795, Validation: 0.796, Test: 0.795, AUC: 0.891
Epoch: 16, Loss: 0.4462, Validation Loss: 0.4420, Train: 0.792, Validation: 0.793, Test: 0.792, AUC: 0.891
Epoch: 17, Loss: 0.4451, Validation Loss: 0.4399, Train: 0.796, Validation: 0.797, Test: 0.795, AUC: 0.891
Epoch: 18, Loss: 0.4441, Validation Loss: 0.4377, Train: 0.800, Validation: 0.801, Test: 0.799, AUC: 0.894
Epoch: 19, Loss: 0.4432, Validation L

KeyboardInterrupt: 